In [1]:
import os

In [2]:
%pwd

'e:\\OneDrive\\Documents\\End-to-End-MLP-with-MLFlow\\research'

Salgamos de Research. Volvamos a la carpeta fuente

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\OneDrive\\Documents\\End-to-End-MLP-with-MLFlow'

Usaré el potencial de los dataclass para ir llamando las rutas 'path' configuradas en mi config.yaml

### hacemos el update de las entities ("entity")

In [6]:
#entity dataclass
from dataclasses import dataclass
from pathlib import Path

# Para crear la folder Entity, importaremos desde esta ruta E:\OneDrive\Documents\End-to-End-MLP-with-MLFlow\src\mlProject\constants\__init__.py qu en si importa desde el config.yaml
@dataclass(frozen=True) #<--- esta entidad, mi dataclass para config ingestion ligado al config.yaml. no es una python class si no que una dataclass. como version escrita de esta funcion.
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

importamos el contrenido en Constants y de utils/comon.py, la función que leerá los yaml con el configBox (read_yaml)

### esto va a configuration
E:\OneDrive\Documents\End-to-End-MLP-with-MLFlow\src\mlProject\config\configuration.py

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

### Update the configuration manager in src config
Esto también lo moveremos a E:\OneDrive\Documents\End-to-End-MLP-with-MLFlow\src\mlProject\config\configuration.py

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig: #<--- llamando desde config.yaml
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

### Update the components
 * cargo librerías para la ingestion de los datos.

In [9]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

### Data Ingestion process
llevamos esto y las rutas del bloque anterior de arriba a components/data_ingestion.py
recordar agregar 'from pathlib import path' y 'from pathlib import Path
from mlProject.entity.config_entity import (DataIngestionConfig)'

en E:\OneDrive\Documents\End-to-End-MLP-with-MLFlow\src\mlProject\components\data_ingestion.py

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    #funcion muy sencilla que descarga la data del URL.
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    # funcion de extracción de los datos del .zip file
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

### Ingesting the Data
* creo el stage01_data_ingestion.py
* en E:\OneDrive\Documents\End-to-End-MLP-with-MLFlow\src\mlProject\pipeline\stage_01_data_ingestion.py

In [ ]:
'''from mlProject.config.configuration import ConfigurationManager
from mlProject.components.data_ingestion import DataIngestion
from mlProject import logger



STAGE_NAME = "Data Ingestion stage"

class DataIngestionTrainingPipeline: #<-- El pipeline que diseñé en mi data_ingestion notebook
    def __init__(self):
        pass

    def main(self): #<-- El main method que llamaré desde main.py en carpeta raiz
        config = ConfigurationManager() #inicio mi ConfigurationManager
        data_ingestion_config = config.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)
        data_ingestion.download_file() #llamo el metodo de download
        data_ingestion.extract_zip_file()# llamo el metodo de extracción


    
if __name__ == '__main__':
    try:
        logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
        obj = DataIngestionTrainingPipeline()
        obj.main()
        logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
    except Exception as e:
        logger.exception(e)
        raise e'''

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [17]:
try:
    config = ConfigurationManager() #inicio mi ConfigurationManager
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file() #llamo el metodo de download
    data_ingestion.extract_zip_file()# llamo el metodo de extracción
except Exception as e:
    raise e

[2023-09-21 21:42:24,644: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-21 21:42:24,645: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-21 21:42:24,647: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-21 21:42:24,648: INFO: common: created directory at: artifacts]
[2023-09-21 21:42:24,649: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-21 21:42:24,650: INFO: 2923246290: File already exists of size: ~ 720 KB]


EL proceso nos devuelve información de la ingestion:
* process detalado sobre su composición,
* tamaño viabilidad del formato de sus datos, 
* interacción con la fuente (Github repo), 
* etc.

Tambien nos muestra la creación del os outputs esperados en la carpeta Artifacts en sus respectivos archivos de formato:
* .zip
* .csv

## Experimentando funcionó bien
* ahora necesitaré tranformar todo este proceso en codigo establecido.
* Esto significa copiar los codeblocks funcionales a sus respectivas folders y files.